In [160]:
#Omid55
# all imports
import numpy as np
import seaborn as sns
from scipy.sparse import csr_matrix

In [161]:
# load data from text edges list (sparse representation of edges)
# output =>
# input  =>
# fname: 
# ...
def load_data(fname, delim, skip_rows_count, time_column_index):
    edges = np.loadtxt(open(fname,"rb"),delimiter=delim,skiprows=skip_rows_count)
    return edges

In [ ]:

def build_unconstrained_network(subset_of_edges, N):
    data = np.ones(len(subset_of_edges))
    row = np.subtract(subset_of_edges[:,0],1)
    col = np.subtract(subset_of_edges[:,1],1)
    A = csr_matrix( (data,(row,col)), shape=(N,N) )
    return A

In [162]:
# process the edges list created from load_data and build evolving networks
# output =>  array of adjacency matrixes
# inputs =>
# edges:         edges list
# window_size:   window size in time not in number of edges
# step_size:     step size (also) in time not in number of edges
def process_data(edges, window_size, step_size):
    N = int(max(edges[:,0].max(), edges[:,1].max()))   # nodes count
    

In [163]:
e = load_data('fb-forum.txt', ',', 0, 2)
#e = load_data('dblp_coauthor/out.dblp_coauthor', ' ', 1, 3)

In [164]:
A = process_data(e,0,0)

In [165]:
print(A.shape)
(A > 0).sum() / (A.shape[0]*A.shape[1])

(899, 899)


0.0087713328738766722